## Importing packages

In [1]:
import random
import time
import pickle
import sys
import re
import string
import collections

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import mpld3

from sklearn import feature_extraction
import gensim
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim

import nltk
from nltk.corpus import stopwords
from nltk.tag.sequential import ClassifierBasedTagger
#from nltk.stem.wordnet import WordNetLemmatizer
#from nltk.tokenize import word_tokenize

import pygraphviz
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout


from IPython.display import display, Image
from IPython.core.interactiveshell import InteractiveShell

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """


## Developing a function to resolve abbreviations

In [2]:
abbreviations1 = pd.read_csv('./csv/fragebogen-abbreviations.csv', 
                             encoding="cp1250",
                             sep=',',
                             usecols=[0, 1], 
                             names=["abbreviated","expanded",])
abbreviations1.dropna(axis=0, inplace=True)
abbreviations1.head(10)

,abbreviated,expanded
0,.ähnl.,ähnlich
1,.Bed.,Bedeutung
2,.Bez.,Bezeichnung
4,.dgl.,dergleichen
5,.Füg.,Fügungen
6,.Komp.,Komposita
7,.Syn.,Synonyme
8,.Untersch.,Unterschied
9,.Vb.,Verba
10,.Volksüberlieferg.,Volksüberlieferung


In [3]:
abbreviations2 = pd.read_csv('./csv/fragebogen-abbreviations.csv', 
                             encoding="cp1250",
                             sep=',',
                             usecols=[2, 3], 
                             names=["abbreviated","expanded",])
abbreviations2.dropna(axis=0, inplace=True)
abbreviations2.head(10)

,abbreviated,expanded
3,.d.,der
20,a.,am
21,A.,Auge
22,ä.,ähnlich
53,Ang.,Angabe
78,B.,Bauer
79,b.,bei
147,d.,der
165,e.,ein
195,F.,Femininum


In [4]:
subst1 = {" " + row["abbreviated"]: " " + row["expanded"] + " " for index, row in abbreviations1.iterrows()}
subst2 = {" " + row["abbreviated"]: " " + row["expanded"] + " " for index, row in abbreviations2.iterrows()}

In [5]:
for key, value in subst1.items():
    print(key, value)
    
    if key == " .Vb.":      #show just the first few items
        break

 .ähnl.  ähnlich 
 .Bed.  Bedeutung 
 .Bez.  Bezeichnung 
 .dgl.  dergleichen 
 .Füg.  Fügungen 
 .Komp.  Komposita 
 .Syn.  Synonyme 
 .Untersch.  Unterschied 
 .Vb.  Verba 


In [6]:
for key, value in subst2.items():
    print(key, value)
    
    if key == " g.":      #show just the first few items
        break

 .d.  der 
 a.  am 
 A.  Auge 
 ä.  ähnlich 
 Ang.  Angabe 
 B.  Bauer 
 b.  bei 
 d.  der 
 e.  ein 
 F.  Femininum 
 f.  für 
 g.  gehen 


## Developing a POS and Lemmatizer for German

#### Getting POS - [training a classifier with the tiger sentence corpus](https://datascience.blog.wzb.eu/2016/07/13/accurate-part-of-speech-tagging-of-german-texts-with-nltk/)

In [7]:
#https://github.com/ptnplanet/NLTK-Contributions/blob/master/ClassifierBasedGermanTagger/ClassifierBasedGermanTagger.py

class ClassifierBasedGermanTagger(ClassifierBasedTagger):
    """A classifier based German part-of-speech tagger. It has an accuracy of
    96.09% after being trained on 90% of the German TIGER corpus. The tagger
    extends the NLTK ClassifierBasedTagger and implements a slightly modified
    feature detector.
    """

    def feature_detector(self, tokens, index, history):
        """Implementing a slightly modified feature detector.
        @param tokens: The tokens from the sentence to tag.
        @param index: The current token index to tag.
        @param history: The previous tagged tokens.
        """

        word = tokens[index]
        if index == 0: # At the beginning of the sentence
            prevword = prevprevword = None
            prevtag = prevprevtag = None
            #word = word.lower() # Lowercase at the beginning of sentence
        elif index == 1:
            prevword = tokens[index-1] # Note: no lowercase
            prevprevword = None
            prevtag = history[index-1]
            prevprevtag = None
        else:
            prevword = tokens[index-1]
            prevprevword = tokens[index-2]
            prevtag = history[index-1]
            prevprevtag = history[index-2]

        if re.match(r'[0-9]+([\.,][0-9]*)?|[0-9]*[\.,][0-9]+$', word):
            # Included "," as decimal point
            shape = 'number'
        elif re.compile(r'\W+$', re.UNICODE).match(word):
            # Included unicode flag
            shape = 'punct'
        elif re.match(r'([A-ZÄÖÜ]+[a-zäöüß]*-?)+$', word):
            # Included dash for dashed words and umlauts
            shape = 'upcase'
        elif re.match(r'[a-zäöüß]+', word):
            # Included umlauts
            shape = 'downcase'
        elif re.compile(r"\w+", re.UNICODE).match(word):
            # Included unicode flag
            shape = 'mixedcase'
        else:
            shape = 'other'

        features = {
            'prevtag': prevtag,
            'prevprevtag': prevprevtag,
            'word': word,
            'word.lower': word.lower(),
            'suffix3': word.lower()[-3:],
            #'suffix2': word.lower()[-2:],
            #'suffix1': word.lower()[-1:],
            'preffix1': word[:1], # included
            'prevprevword': prevprevword,
            'prevword': prevword,
            'prevtag+word': '%s+%s' % (prevtag, word),
            'prevprevtag+word': '%s+%s' % (prevprevtag, word),
            'prevword+word': '%s+%s' % (prevword, word),
            'shape': shape
            }
        return features

#### Training with all sentences

In [8]:
with open('./nlp/nltk_german_classifier_data.pickle', 'rb') as f:
    tiger_tagger = pickle.load(f)

In [9]:
tiger_tagger.tag(['Das'])[0]

('Das', 'ART')

#### Strategy 1 for lemma - [Reading the Lemmata from the tiger sentence corpus](https://datascience.blog.wzb.eu/2016/07/13/accurate-part-of-speech-tagging-of-german-texts-with-nltk/)

In [10]:
def read_lemmata_from_tiger_corpus(tiger_corpus_file, valid_cols_n=15, col_words=1, col_lemmata=2):
    lemmata_mapping = {}

    with open(tiger_corpus_file, encoding="utf8") as f:
        for line in f:
            parts = line.split()
            if len(parts) == valid_cols_n:
                w, lemma = parts[col_words], parts[col_lemmata]
                if w != lemma and w not in lemmata_mapping and not lemma.startswith('--'):
                    lemmata_mapping[w] = lemma

    return lemmata_mapping

lemmata_mapping = read_lemmata_from_tiger_corpus('./nlp/tiger_release_aug07.corrected.16012013.conll09')

In [11]:
lemmata_mapping['Das']

'der'

#### Strategy 2 for lemma - If we have the POS, we can use the [German Lemmatizer](https://github.com/WZBSocialScienceCenter/germalemma)  
('N...' (nouns), 'V...' (verbs), 'ADJ...' (adjectives), 'ADV...' (adverbs)


In [12]:
#https://github.com/WZBSocialScienceCenter/germalemma
#https://datascience.blog.wzb.eu/2016/07/13/accurate-part-of-speech-tagging-of-german-texts-with-nltk/
#https://datascience.blog.wzb.eu/2016/07/13/autocorrecting-misspelled-words-in-python-using-hunspell/
from germalemma import GermaLemma

# passing the word and the POS tag ("N" for noun)
lemmatizer = GermaLemma()

def germallema(word, pos):
    pos = tiger_tagger.tag([word])[0][1]
    lemma = lemmatizer.find_lemma(word, pos)
    return(lemma)

In [13]:
germallema("Arme", tiger_tagger.tag(["Arme"])[0][1])

'Arm'

#### Strategy 4 - Using [hunspell](https://stackoverflow.com/questions/53029568/how-do-i-install-hunspell-python-package) for a fallback [option](https://datascience.blog.wzb.eu/2016/07/13/autocorrecting-misspelled-words-in-python-using-hunspell/)

#### Using all the resources together:

In [14]:
def robust_lemmatization(word):
    lemma = None
    try:
        pos = tiger_tagger.tag([word])[0][1]
        lemma = germallema(word, pos)
    except:
        pass
    #if not lemma:
    #    try:
    #        lemma = hunspell_lemma(word)
    #    except:
    #        pass
    if not lemma:
        try:
            lemma = lemmata_mapping[word]
        except:
            pass
    if lemma:
        return(lemma)
    else:
        return(word)

In [15]:
robust_lemmatization("Arme")

'Arm'

## 1 - Fragebogen

#### Reading fragebogen information

In [16]:
fragebogen = pd.read_csv('./csv/fragebogen.csv', names=["id",
                                                        "nummer", 
                                                        "titel", 
                                                        "schlagwoerter", 
                                                        "erscheinungsjahr",
                                                        "autoren", 
                                                        "originaldaten",
                                                        "anmerkung",
                                                        "freigabe",
                                                        "checked",
                                                        "wordleiste",
                                                        "druck",
                                                        "online",
                                                        "publiziert",
                                                        "fragebogen_typ_id",])

fragebogen.drop(["id","schlagwoerter","erscheinungsjahr","autoren","originaldaten","anmerkung","freigabe",
                 "checked","wordleiste","druck","online","publiziert","fragebogen_typ_id",], inplace=True, axis=1)

fragebogen.set_index("nummer", drop=True, inplace=True)

In [17]:
fragebogen.info()

<class 'pandas.core.frame.DataFrame'>
Index: 762 entries, 1 to ETir.28
Data columns (total 1 columns):
titel    762 non-null object
dtypes: object(1)
memory usage: 11.9+ KB


In [18]:
fragebogen.head(40)

,titel
nummer,
1,Fragebogen 1: Kopf (1)
2,Fragebogen 2: Die Osterwoche (1)
3,Fragebogen 3: Die Osterwoche (2)
4,Fragebogen 4: Kopf (2)
5,Fragebogen 5: Zeit zwischen Ostern und Fronlei...
6,Fragebogen 6: Menschl. Haar und Bart (= Kopf 3)
7,Fragebogen 7: Hochzeit (1)
8,Fragebogen 8: Hochzeit (2)
9,Fragebogen 9: Hochzeit (3)


#### Filtering the original fragebogen from the CSV file

In [19]:
fragebogen.tail()

,titel
nummer,
ETir.24,24. Ergänzungsfragebogen zum Tiroler Fragebuch
ETir.25,25. Ergänzungsfragebogen zum Tiroler Fragebuch
ETir.26,26. Ergänzungsfragebogen zum Tiroler Fragebuch
ETir.27,27. Ergänzungsfragebogen zum Tiroler Fragebuch
ETir.28,28. Ergänzungsfragebogen zum Tiroler Fragebuch


In [20]:
fragebogen[fragebogen.titel.str.startswith('Fragebogen')].tail()

,titel
nummer,
105,Fragebogen 105: Verkehr
106,Fragebogen 106: Wasser
107,Fragebogen 107: Gewerbe
108,Fragebogen 108: Gewerbe
109,Fragebogen 109: Gewerbe


In [21]:
fb_original = fragebogen[fragebogen.titel.str.startswith('Fragebogen')]

#### Extracting information from the fragebogen titel

In [22]:
regex1 = r'([Fragebon]+)\s{1}([0-9]+)[:]{1}([,A-ZÄÖÜa-zäöüß0-9.\s]+)[,\s]*([=\-\(\)\sA-ZÄÖÜa-zäöüß0-9]*)'

fb_original.titel.str.extract(regex1).head()
#fb_original.titel.str.extract(regex1).tail()

,0,1,2,3
nummer,,,,
1,Fragebogen,1,Kopf,(1)
2,Fragebogen,2,Die Osterwoche,(1)
3,Fragebogen,3,Die Osterwoche,(2)
4,Fragebogen,4,Kopf,(2)
5,Fragebogen,5,Zeit zwischen Ostern und Fronleichnam,


In [23]:
fb_original['number'] = fb_original.titel.str.extract(regex1)[1].str.strip()
fb_original['headwords'] = fb_original.titel.str.extract(regex1)[2].str.strip()
fb_original['series'] = fb_original.titel.str.extract(regex1)[3].str.strip()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

#### Expanding abbreviations, tokenizing, eliminating stopwords and lemmatizing

In [24]:
my_stopwords = set(["i", "ii"])
german_stopwords = set(stopwords.words('german')).union(my_stopwords)

In [25]:
#https://stackoverflow.com/questions/35231285/python-how-to-split-a-string-by-non-alpha-characters

def expand_abbreviations(text):
    for key in subst1.keys():
        if key in text:
            text = text.replace(key, subst1[key])
            changes["abbreviation: "+key] = subst1[key]
        if key.lower() in text:
            text = text.replace(key.lower(), subst1[key])
            changes["abbreviation: "+key.lower()] = subst1[key]
    for key in subst2.keys():
        if key in text:
            text = text.replace(key, subst2[key])
            changes["abbreviation: "+key] = subst2[key]
        if key.lower() in text:
            text = text.replace(key.lower(), subst2[key])
            changes["abbreviation: "+key.lower()] = subst2[key]
    return(text)        

def lemmatize(word):
    lemma = robust_lemmatization(word)
    lemma = lemma.lower()
    if word.lower() != lemma.lower():
        changes["lemma: "+word] = lemma
        #print("Lemma: changed {} for {}".format(word, lemma))
    return(lemma)


def expand_tokenize_clean_lemmatize(input_text):
    output = re.sub(r'/',  ' ', input_text)
    output = re.sub(r'\(',  ' ', output)
    output = re.sub(r'\)',  ' ', output)
    output = expand_abbreviations(output)
    output = re.findall(r'\w+', output)
    output = [elem.strip(string.punctuation) for elem in output]
    output = [lemmatize(elem) for elem in output]
    output = [elem.lower() for elem in output if (elem.lower() not in german_stopwords)]
    output = [elem for elem in output if not elem.isnumeric()]
    return output

In [26]:
changes = {}
fb_original['headwords_list'] = fb_original.headwords.apply(expand_tokenize_clean_lemmatize)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [27]:
for key, value in sorted(changes.items()):
    print(key, "----> ", value)

lemma: Arbeiten ---->  arbeit
lemma: Das ---->  der
lemma: Die ---->  der
lemma: Feldfrüchte ---->  feldfrucht
lemma: Gewichte ---->  gewicht
lemma: Gliedmaßen ---->  gliedmaß
lemma: Hautkrankheiten ---->  hautkrankheit
lemma: Körperteile ---->  körperteil
lemma: Maße ---->  maß
lemma: Tiere ---->  tier
lemma: Weibliche ---->  weiblich
lemma: ihre ---->  ihr
lemma: unserer ---->  unser


In [28]:
fb_original.head(10)

,titel,number,headwords,series,headwords_list
nummer,,,,,
1,Fragebogen 1: Kopf (1),1,Kopf,(1),[kopf]
2,Fragebogen 2: Die Osterwoche (1),2,Die Osterwoche,(1),[osterwoche]
3,Fragebogen 3: Die Osterwoche (2),3,Die Osterwoche,(2),[osterwoche]
4,Fragebogen 4: Kopf (2),4,Kopf,(2),[kopf]
5,Fragebogen 5: Zeit zwischen Ostern und Fronlei...,5,Zeit zwischen Ostern und Fronleichnam,,"[zeit, ostern, fronleichnam]"
6,Fragebogen 6: Menschl. Haar und Bart (= Kopf 3),6,Menschl. Haar und Bart,(= Kopf 3),"[menschl, haar, bart]"
7,Fragebogen 7: Hochzeit (1),7,Hochzeit,(1),[hochzeit]
8,Fragebogen 8: Hochzeit (2),8,Hochzeit,(2),[hochzeit]
9,Fragebogen 9: Hochzeit (3),9,Hochzeit,(3),[hochzeit]


In [29]:
#print(fb_original.index)
print(fb_original.number.unique())
print()
print(set(fb_original.headwords_list.sum()))
print()
print(fb_original.series.unique())

['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57' '58'
 '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71' '72'
 '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86'
 '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99' '100'
 '101' '102' '103' '104' '105' '106' '107' '108' '109']

{'gewerbe', 'ostern', 'dank', 'schuster', 'gewicht', 'unterricht', 'ohr', 'farbe', 'mittsommer', 'forstwirtschaft', 'wiese', 'speisen', 'menschl', 'erdoberfläche', 'flur', 'verwertung', 'körper', 'geselligkeit', 'imkerei', 'trinken', 'besiedelung', 'heuarbeit', 'liebe', 'mythologisches', 'gruß', 'verkehr', 'hund', 'weißgebäck', 'sehen', 'eßzeug', 'weinbau', 'freundschaft', 'fluren', 'katze', 'licht', 'wochenbett', 'wunsch', 'geflügel', 'wei

In [30]:
fb_original.groupby("headwords")["number"].apply(lambda group_series: group_series.tolist()).reset_index().head(30)

,headwords,number
0,1. Erdoberfläche,[104]
1,Arm,[33]
2,Auge,[13]
3,"Besiedelung, Flur","[14, 15]"
4,Bewegung,"[42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]"
5,Bierbrauerei,[101]
6,"Blutsverwandtschaft, Altersstufen",[87]
7,Brotbacken,"[28, 29, 30]"
8,Das Pferd,[76]
9,Der Bauernhof,"[92, 93]"


In [31]:
#https://matplotlib.org/2.0.2/api/colors_api.html

G = nx.Graph()

for concept_list in fb_original.headwords_list:
    for concept in concept_list:
        G.add_node(concept, color='white')
        
for questionnaire in fb_original.number:
    G.add_node(questionnaire, color='yellow')
    
for idn in fb_original.index:
    for concept in fb_original.loc[idn, 'headwords_list']:
        G.add_edge(concept, fb_original.loc[idn, 'number'], length = 10)

In [32]:
#print(G.nodes)
print(G.edges)

[('kopf', '1'), ('kopf', '4'), ('osterwoche', '2'), ('osterwoche', '3'), ('zeit', '5'), ('zeit', '54'), ('zeit', '68'), ('zeit', '72'), ('ostern', '5'), ('ostern', '72'), ('fronleichnam', '5'), ('menschl', '6'), ('haar', '6'), ('bart', '6'), ('hochzeit', '7'), ('hochzeit', '8'), ('hochzeit', '9'), ('hochzeit', '10'), ('hochzeit', '11'), ('nase', '12'), ('ohr', '12'), ('auge', '13'), ('besiedelung', '14'), ('besiedelung', '15'), ('flur', '14'), ('flur', '15'), ('feld', '16'), ('feld', '17'), ('feld', '18'), ('feld', '19'), ('feldbestellung', '16'), ('feldbestellung', '17'), ('feldbestellung', '18'), ('feldbestellung', '19'), ('körper', '20'), ('körper', '21'), ('körper', '22'), ('körper', '23'), ('körperteil', '24'), ('körperteil', '77'), ('körperteil', '78'), ('hautkrankheit', '25'), ('hautkrankheit', '26'), ('hautkrankheit', '27'), ('brotbacken', '28'), ('brotbacken', '29'), ('brotbacken', '30'), ('weißgebäck', '31'), ('gliedmaß', '32'), ('arm', '32'), ('arm', '33'), ('hand', '34'), (

In [33]:
#https://stackoverflow.com/questions/48240021/alter-edge-length-and-cluster-spacing-in-networkx-matplotlib-force-graph
#if Win: https://graphviz.gitlab.io/_pages/Download/Download_windows.html
# https://stackoverflow.com/questions/4571067/installing-pygraphviz-on-windows?rq=1
#if Anaconda: conda install graphviz pygraphviz -c alubbock
#pip install --global-option=build_ext --global-option="-IC:\Program Files\<path>\Graphviz2.38\include" --global-option="-LC:\Program Files\Graphviz2.38\lib\release\lib" pygraphviz

plt.figure(figsize=(14,18))

#pos = nx.spring_layout(G)
#pos = graphviz_layout(G)
pos = graphviz_layout(G, prog='twopi', args='')


nx.draw(G, 
        pos,
        #width=5,
        alpha=0.9,
        #cmap=plt.get_cmap('viridis'),
        node_color=nx.get_node_attributes(G,'color').values(), 
        with_labels=True,
        font_weight='bold', 
        #font_color='white',
       )


plt.show()

ValueError: Program twopi not found in path.

<Figure size 1008x1296 with 0 Axes>

## 2 - Frage

#### Reading frage information

In [33]:
frage = pd.read_csv('./csv/frage.csv', names=["id", "fragebogen_id","nummer", "original_frage", "kurzfrage", "originaldaten"])

In [34]:
frage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24382 entries, 0 to 24381
Data columns (total 6 columns):
id                24382 non-null int64
fragebogen_id     24382 non-null int64
nummer            24382 non-null object
original_frage    24382 non-null object
kurzfrage         17413 non-null object
originaldaten     6528 non-null object
dtypes: int64(2), object(4)
memory usage: 1.1+ MB


#### Filtering the relevant columns and the original questionnaires

In [35]:
frage.drop(["id", "originaldaten", "kurzfrage"], axis=1, inplace=True)
frage = frage[frage.fragebogen_id <= 109]
frage.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16744 entries, 0 to 16743
Data columns (total 3 columns):
fragebogen_id     16744 non-null int64
nummer            16744 non-null object
original_frage    16744 non-null object
dtypes: int64(1), object(2)
memory usage: 523.2+ KB


In [36]:
frage.head()

,fragebogen_id,nummer,original_frage
0,1,A1,"Kopf: Kopf, Haupt; auch scherzh./übertr."
1,1,A2,Kopf: Kopf/Haupt (in urspr. Bed.) in Vergl./Ra...
2,1,A3,Kopf: Kopf/Haupt (übertr.) in Vergl./Ra. (das ...
3,1,A4,"Kopf: schöner, ebenmäßig gebauter Kopf (Christ..."
4,1,A4a,"Kopf: häßlicher, unebenmäßiger Kopf"


In [37]:
frage.tail()

,fragebogen_id,nummer,original_frage
16739,109,602,Gewerbe: (Gehalts-)Zulage
16740,109,603,"Gewerbe: Abzug v. Gehalt, Pl."
16741,109,604,"Gewerbe: sonstige heute noch lebende, bodenstä..."
16742,109,605,"Gewerbe: abgekommene, bodenständige Berufe, di..."
16743,109,606,"Gewerbe: bodenständige FN, die vermutl. auf ei..."


#### Extracting information from the frage "original_frage" field

In [81]:
#regex2 = r'([A-ZÄÖÜa-zäöüß.\(\)\-\/]+)[:]{1}([A-ZÄÖÜa-zäöüß0-9=,;:\-\(\)\.\?\s]+)'
#regex2 = r'([A-ZÄÖÜa-zäöüß.\(\)\-\/]+)[:]{1}([A-ZÄÖÜa-zäöüß0-9=,;:\/\-\(\)\.\?\s]+)'
regex2 = r'([A-ZÄÖÜa-zäöüß.\(\)\-\/]+)[:]{1}(.+)'

frage.original_frage.str.extract(regex2).head(10)

,0,1
0,Kopf,"Kopf, Haupt; auch scherzh./übertr."
1,Kopf,Kopf/Haupt (in urspr. Bed.) in Vergl./Ra. (Ko...
2,Kopf,Kopf/Haupt (übertr.) in Vergl./Ra. (das ist e...
3,Kopf,"schöner, ebenmäßig gebauter Kopf (Christuskop..."
4,Kopf,"häßlicher, unebenmäßiger Kopf"
5,Kopf,großer Kopf; großkopfig
6,Kopf,Wasserkopf
7,Kopf,kleiner Kopf
8,Kopf,oben zugespitzter Kopf
9,Kopf,langer Kopf


In [82]:
frage['main_theme'] = frage.original_frage.str.extract(regex2)[0]
frage.dropna(subset=['main_theme'], inplace=True,)
frage['question_words'] = frage.original_frage.str.extract(regex2)[1].str.strip()

In [83]:
frage.head()

,fragebogen_id,nummer,original_frage,main_theme,question_words
0,1,A1,"Kopf: Kopf, Haupt; auch scherzh./übertr.",Kopf,"Kopf, Haupt; auch scherzh./übertr."
1,1,A2,Kopf: Kopf/Haupt (in urspr. Bed.) in Vergl./Ra...,Kopf,Kopf/Haupt (in urspr. Bed.) in Vergl./Ra. (Kop...
2,1,A3,Kopf: Kopf/Haupt (übertr.) in Vergl./Ra. (das ...,Kopf,Kopf/Haupt (übertr.) in Vergl./Ra. (das ist ei...
3,1,A4,"Kopf: schöner, ebenmäßig gebauter Kopf (Christ...",Kopf,"schöner, ebenmäßig gebauter Kopf (Christuskopf..."
4,1,A4a,"Kopf: häßlicher, unebenmäßiger Kopf",Kopf,"häßlicher, unebenmäßiger Kopf"


#### Expanding abbreviations, tokenizing, eliminating stopwords and lemmatizing¶

In [84]:
changes = {}
frage['main_theme'] = frage['main_theme'].apply(expand_tokenize_clean_lemmatize)

In [85]:
for key, value in sorted(changes.items()):
    print(key, "----> ", value)

abbreviation:  Aufbahrg. ---->   Aufbahrung 
abbreviation:  Bewegg. ---->   Bewegung 
abbreviation:  Bw. ---->   Bestimmungswort 
abbreviation:  Eigensch. ---->   Eigenschaften 
abbreviation:  Gw. ---->   Grundwort 
abbreviation:  Herstellg. ---->   Herstellung 
abbreviation:  Kleidg. ---->   Kleidung 
abbreviation:  Krankh. ---->   Krankheit 
abbreviation:  Vkdl. ---->   Volkskundliches 
abbreviation:  m. ---->   mit 
abbreviation:  spött. ---->   spöttisch 
abbreviation:  undeutl. ---->   undeutlich 
abbreviation:  zs. ---->   zusammen 
lemma: Adern ---->  ader
lemma: Buche ---->  buch
lemma: Eigenschaften ---->  eigenschaft
lemma: Enten ---->  ente
lemma: Erwachsener ---->  erwachsene
lemma: Feldfrüchte ---->  feldfrucht
lemma: Fische ---->  fisch
lemma: Gewichte ---->  gewicht
lemma: Gliedmaßen ---->  gliedmaß
lemma: Heilige ---->  heilig
lemma: Insekten ---->  insekt
lemma: Kinder ---->  kind
lemma: Körperteile ---->  körperteil
lemma: Mahlzeiten ---->  mahlzeit
lemma: Maße ----> 

In [86]:
changes = {}
frage['question_words'] = frage.question_words.apply(expand_tokenize_clean_lemmatize)

In [87]:
for key, value in sorted(changes.items()):
    print(key, "----> ", value)

abbreviation:  A. ---->   Auge 
abbreviation:  ASg. ---->   Akkusativ_Singular 
abbreviation:  Abb. ---->   Abbildung 
abbreviation:  Abl. ---->   Ableitungen 
abbreviation:  Ableitg. ---->   Ableitung  
abbreviation:  Abschuppg. ---->   Abschuppung 
abbreviation:  Abschwächg. ---->   Abschwächung 
abbreviation:  Abteilg. ---->   Abteilung 
abbreviation:  Abweisg. ---->   Abweisung 
abbreviation:  Abzahlg. ---->   Abzahlung 
abbreviation:  Ackerbez. ---->   Ackerbezeichnungen 
abbreviation:  Ackerg. ---->   Ackerung 
abbreviation:  Adj. ---->   Adjektiv 
abbreviation:  Adv. ---->   Adverbien 
abbreviation:  Allgemeinbez. ---->   Allgemeinbezeichnung 
abbreviation:  Almwirtsch. ---->   Almwirtschaft 
abbreviation:  Altersversorgg. ---->   Altersversorgung 
abbreviation:  Ammenspr. ---->   Ammensprache 
abbreviation:  Ang. ---->   Angabe 
abbreviation:  Anordng. ---->   Anordnung 
abbreviation:  Anwendg. ---->   Anwendung 
abbreviation:  Art. ---->   Artikel 
abbreviation:  Artikelgebr. 

lemma: Männer ---->  mann
lemma: Männerbünde ---->  männerbund
lemma: Männern ---->  mann
lemma: Märkten ---->  markt
lemma: Mühlen ---->  mühle
lemma: Münzen ---->  münze
lemma: Nachbarn ---->  nachbar
lemma: Nachbarsleute ---->  nachbarsleut
lemma: Nachfahren ---->  nachfahre
lemma: Nahen ---->  nah
lemma: Namen ---->  name
lemma: Narben ---->  narbe
lemma: Narren ---->  narr
lemma: Nerven ---->  nerv
lemma: Netze ---->  netz
lemma: Netzen ---->  netz
lemma: Neue ---->  neu
lemma: Neuerungen ---->  neuerung
lemma: Neureute ---->  neureut
lemma: Nieren ---->  niere
lemma: Noten ---->  note
lemma: Nummern ---->  nummer
lemma: Nächte ---->  nacht
lemma: Objekte ---->  objekt
lemma: Ohren ---->  ohr
lemma: Opfern ---->  opfer
lemma: Orte ---->  ort
lemma: Orten ---->  ort
lemma: Ortes ---->  ort
lemma: Paare ---->  paar
lemma: Perlen ---->  perle
lemma: Personen ---->  person
lemma: Pfeifen ---->  pfeife
lemma: Pferde ---->  pferd
lemma: Pferden ---->  pferd
lemma: Pfieselknechte ---->  

lemma: austreiben ---->  austreib
lemma: austreten ---->  austret
lemma: austretten ---->  austrett
lemma: austräger ---->  austräg
lemma: ausverkaufen ---->  ausverkauf
lemma: auswarten ---->  auswart
lemma: auswaschen ---->  auswasch
lemma: ausweisen ---->  ausweis
lemma: auswendiger ---->  auswendig
lemma: auswerchen ---->  auswerch
lemma: auswerfen ---->  auswerf
lemma: auswischen ---->  auswisch
lemma: auswürchen ---->  auswürch
lemma: ausziehen ---->  auszieh
lemma: auszudrücken ---->  ausdrücken
lemma: auszugleichen ---->  auszugleich
lemma: außerhaben ---->  außerhab
lemma: bacher ---->  bach
lemma: baldige ---->  baldig
lemma: bampfen ---->  bampf
lemma: band ---->  binden
lemma: barchentene ---->  barchenten
lemma: barkasse ---->  barkassen
lemma: barte ---->  baren
lemma: bartet ---->  baren
lemma: batterie ---->  batteri
lemma: batziges ---->  batzig
lemma: bauchiger ---->  bauchig
lemma: bauer ---->  bau
lemma: baufälliges ---->  baufällig
lemma: baumen ---->  baum
lemma: 

lemma: fertiger ---->  fertig
lemma: fertiges ---->  fertig
lemma: fescher ---->  fesch
lemma: fesches ---->  fesch
lemma: festanliegende ---->  festanliegend
lemma: feste ---->  fest
lemma: festen ---->  fest
lemma: fester ---->  fest
lemma: festes ---->  fest
lemma: festgebunden ---->  festgebund
lemma: festgelaufenes ---->  festgelaufen
lemma: festgewordene ---->  festgeworden
lemma: festgewordener ---->  festgeworden
lemma: festlaufen ---->  festlauf
lemma: festlicher ---->  festlich
lemma: festsitzende ---->  festsitzend
lemma: feststehendes ---->  feststehend
lemma: festzuhalten ---->  festzuhalt
lemma: fette ---->  fett
lemma: fettes ---->  fett
lemma: feuchte ---->  feucht
lemma: feuchten ---->  feucht
lemma: feuchtes ---->  feucht
lemma: feuer ---->  feu
lemma: feuerfeste ---->  feuerfest
lemma: feurige ---->  feurig
lemma: feuriger ---->  feurig
lemma: fichtenes ---->  fichten
lemma: fiesler ---->  fiesl
lemma: fillen ---->  fill
lemma: findet ---->  finden
lemma: finsterer -

lemma: gruben ---->  grub
lemma: grummeten ---->  grummet
lemma: grunen ---->  grun
lemma: gruner ---->  grun
lemma: grunzen ---->  grunz
lemma: gräteter ---->  grätet
lemma: grätschen ---->  grätsch
lemma: gröberes ---->  gröber
lemma: gröbste ---->  gröbst
lemma: gröllen ---->  gröll
lemma: größer ---->  groß
lemma: größere ---->  groß
lemma: größeren ---->  groß
lemma: größerer ---->  groß
lemma: gründliche ---->  gründlich
lemma: grüne ---->  grün
lemma: grünen ---->  grün
lemma: grüner ---->  grün
lemma: grünes ---->  grün
lemma: grünige ---->  grünig
lemma: grüßen ---->  grüß
lemma: gschwirige ---->  gschwirig
lemma: guche ---->  guch
lemma: guckessen ---->  guckess
lemma: guckt ---->  gucken
lemma: gufe ---->  guf
lemma: gulden ---->  guld
lemma: gumpen ---->  gump
lemma: gumpiger ---->  gumpig
lemma: gupfen ---->  gupf
lemma: gurren ---->  gurr
lemma: gurretzen ---->  gurretz
lemma: gurtnen ---->  gurtn
lemma: gutartiger ---->  gutartig
lemma: gute ---->  gut
lemma: gutem ---->

lemma: lästiger ---->  lästig
lemma: läufer ---->  läuf
lemma: läuft ---->  laufen
lemma: läuse ---->  läus
lemma: läutet ---->  läuten
lemma: läßt ---->  lassen
lemma: låsser ---->  låss
lemma: löcher ---->  löch
lemma: lösche ---->  lösch
lemma: löschen ---->  lösch
lemma: löst ---->  lösen
lemma: löten ---->  löt
lemma: lücke ---->  lück
lemma: lügt ---->  lügen
lemma: lünden ---->  lünd
lemma: lüpfen ---->  lüpf
lemma: mach ---->  machen
lemma: macher ---->  mach
lemma: macht ---->  machen
lemma: made ---->  mad
lemma: mag ---->  mögen
lemma: magere ---->  mager
lemma: magerer ---->  mager
lemma: mahder ---->  mahd
lemma: mahlt ---->  malen
lemma: mange ---->  mang
lemma: mannetes ---->  mannet
lemma: mannstoll ---->  mannstollen
lemma: marken ---->  mark
lemma: markiert ---->  markieren
lemma: markierte ---->  markiert
lemma: markten ---->  markt
lemma: maschinellen ---->  maschinell
lemma: maschinstricken ---->  maschinstrick
lemma: masken ---->  mask
lemma: maskierte ---->  mask

lemma: strickt ---->  stricken
lemma: striemen ---->  striem
lemma: stroachen ---->  stroach
lemma: strohen ---->  stroh
lemma: stromartiger ---->  stromartig
lemma: stromen ---->  strom
lemma: strotten ---->  strott
lemma: struppiger ---->  struppig
lemma: sträubendes ---->  sträubend
lemma: sträubt ---->  sträuben
lemma: strömen ---->  ström
lemma: strütten ---->  strütt
lemma: studierter ---->  studiert
lemma: stufen ---->  stuf
lemma: stumpfe ---->  stumpf
lemma: stunde ---->  stund
lemma: stupfen ---->  stupf
lemma: stute ---->  stut
lemma: städtische ---->  städtisch
lemma: städtischer ---->  städtisch
lemma: ständer ---->  ständ
lemma: stänger ---->  stäng
lemma: stärker ---->  stark
lemma: stäubchen ---->  stäubch
lemma: stäuben ---->  stäub
lemma: stöhnt ---->  stöhnen
lemma: störrische ---->  störrisch
lemma: stößt ---->  stoßen
lemma: stüpfer ---->  stüpf
lemma: stürmen ---->  stürm
lemma: stürzt ---->  stürzen
lemma: substantivierte ---->  substantivieren
lemma: sucht ---->

lemma: zuherführen ---->  zuherführ
lemma: zuherkommen ---->  zuherkomm
lemma: zuherreisen ---->  zuherreis
lemma: zuhinfahren ---->  zuhinfahr
lemma: zuhinführen ---->  zuhinführ
lemma: zuhinkommen ---->  zuhinkomm
lemma: zuhinmähen ---->  zuhinmäh
lemma: zuhören ---->  zuhör
lemma: zukehren ---->  zukehr
lemma: zukommen ---->  zukomm
lemma: zulage ---->  zulag
lemma: zulassen ---->  zulass
lemma: zulaufen ---->  zulauf
lemma: zuleibe ---->  zuleib
lemma: zum ---->  zu
lemma: zumachen ---->  zumach
lemma: zumessen ---->  zumess
lemma: zunehmen ---->  zunehm
lemma: zunähen ---->  zunäh
lemma: zupeilen ---->  zupeil
lemma: zupfen ---->  zupf
lemma: zur ---->  zu
lemma: zurechtweisen ---->  zurechtweis
lemma: zureden ---->  zured
lemma: zureiben ---->  zureib
lemma: zureichen ---->  zureich
lemma: zuriesen ---->  zuries
lemma: zurisen ---->  zuris
lemma: zurufen ---->  zuruf
lemma: zurückbleibt ---->  zurückbleiben
lemma: zurückfliegt ---->  zurückfliegen
lemma: zurückführen ---->  zurüc

In [88]:
frage.head(10)
#frage.info()

,fragebogen_id,nummer,original_frage,main_theme,question_words
0,1,A1,"Kopf: Kopf, Haupt; auch scherzh./übertr.",[kopf],"[kopf, haupt, scherzhaft, übertragen]"
1,1,A2,Kopf: Kopf/Haupt (in urspr. Bed.) in Vergl./Ra...,[kopf],"[kopf, haupt, ursprünglich, bedeutung, verglei..."
2,1,A3,Kopf: Kopf/Haupt (übertr.) in Vergl./Ra. (das ...,[kopf],"[kopf, haupt, übertragen, vergleich, redensart..."
3,1,A4,"Kopf: schöner, ebenmäßig gebauter Kopf (Christ...",[kopf],"[schön, ebenmäßig, gebaut, kopf, christuskopf,..."
4,1,A4a,"Kopf: häßlicher, unebenmäßiger Kopf",[kopf],"[häßlich, unebenmäßig, kopf]"
5,1,A5,Kopf: großer Kopf; großkopfig,[kopf],"[groß, kopf, großkopfig]"
6,1,A5a,Kopf: Wasserkopf,[kopf],[wasserkopf]
7,1,A6,Kopf: kleiner Kopf,[kopf],"[klein, kopf]"
8,1,A7,Kopf: oben zugespitzter Kopf,[kopf],"[oben, zugespitzt, kopf]"
9,1,A8,Kopf: langer Kopf,[kopf],"[lang, kopf]"


#### Examining the common concepts

In [89]:
concepts = []
for entry in frage.question_words.values:  
    for concept in entry:
        concepts.append(concept)
concepts = [c for c in concepts if c not in german_stopwords]
fdconcepts = collections.Counter(concepts)
fdconcepts.most_common(100)

[('redensarten', 1388),
 ('komp', 1170),
 ('beispiel', 1011),
 ('bedeutung', 943),
 ('plural', 707),
 ('komposita', 540),
 ('deminutiv', 480),
 ('femininum', 433),
 ('ausdrücke', 402),
 ('wer', 323),
 ('besonderer', 311),
 ('ähnlich', 248),
 ('bauer', 240),
 ('verb', 234),
 ('gehen', 234),
 ('name', 228),
 ('kommen', 219),
 ('bezeichnung', 216),
 ('adjektiv', 215),
 ('ab', 193),
 ('wendungen', 191),
 ('gut', 191),
 ('teil', 186),
 ('sonstig', 185),
 ('form', 185),
 ('synonyme', 166),
 ('heißen', 159),
 ('übertragen', 156),
 ('art', 151),
 ('üblich', 150),
 ('hand', 149),
 ('lassen', 148),
 ('mensch', 142),
 ('auge', 140),
 ('geschlecht', 138),
 ('sei', 135),
 ('groß', 132),
 ('klein', 131),
 ('geben', 129),
 ('kind', 127),
 ('holz', 126),
 ('füg', 125),
 ('wann', 123),
 ('bräuche', 118),
 ('ra', 118),
 ('volksglaube', 115),
 ('brot', 115),
 ('stelle', 112),
 ('dergleichen', 112),
 ('abl', 112),
 ('alt', 111),
 ('ableitungen', 109),
 ('verwendung', 107),
 ('bedeuten', 103),
 ('wort', 10

In [91]:
#anomalie = 'ab'
anomalie = 'ra'
#anomalie = 'b'

for elem in frage[frage.question_words.apply(lambda x: anomalie in x)][['original_frage','question_words']][0:5].values:
    print(elem)
    print()

["Karwoche: Ra. zu 'Mette': besoffene Metten, jemanden ausmettnen = auszanken"
 list(['ra', 'mette', 'besoffen', 'metten', 'jemand', 'ausmettnen', 'auszanken'])]

['Kopf: Ra. über d. Weisheitszahn' list(['ra', 'weisheitszahn'])]

['Haar: Ra./Sprichwörter, auf Haar bezügl.'
 list(['ra', 'sprichwörter', 'haar', 'bezüglich'])]

['Bart: Ra., Sprichw., d. Bart betreffend'
 list(['ra', 'sprichworte', 'bart', 'betreffend'])]

["Hochzeit: Ra. zu heiraten, wie 'wir sind nicht miteinander verheiratet' u.a.; juristisch: 'auf die Jahre heiraten'"
 list(['ra', 'heirat', 'miteinander', 'verheiraten', 'juristisch', 'jahr', 'heirat'])]



#### (Sanity checking)

In [92]:
re.findall(regex2, "Karwoche: Ra. zu 'Mette': besoffene Metten, jemanden ausmettnen = auszanken")

[('Karwoche',
  " Ra. zu 'Mette': besoffene Metten, jemanden ausmettnen = auszanken")]

In [93]:
expand_tokenize_clean_lemmatize("Hochzeit: Ra. zu heiraten, wie 'wir sind nicht miteinander verheiratet' u.a.; juristisch: 'auf die Jahre heiraten'")

['hochzeit',
 'redensarten',
 'heirat',
 'miteinander',
 'verheiraten',
 'juristisch',
 'jahr',
 'heirat']

#### Drawing the Graphs of specific Fragebogen / Frage concepts

In [ ]:
filtered = frage[frage.fragebogen_id <= 4]

In [ ]:
G = nx.Graph()

for concept_list in filtered.question_words:
    for concept in concept_list:
        G.add_node(concept, color='white')
        
for questionnaire in filtered.fragebogen_id:
    G.add_node(questionnaire, color='yellow')
    
for idn in filtered.index:
    for concept in filtered.loc[idn, 'question_words']:
        G.add_edge(concept, filtered.loc[idn, 'fragebogen_id'], length = 10)

In [ ]:
plt.figure(figsize=(28,28))

#pos = nx.spring_layout(G)
#pos = graphviz_layout(G)
pos = graphviz_layout(G, prog='twopi', args='')


nx.draw(G, 
        pos,
        #width=5,
        alpha=0.9,
        #cmap=plt.get_cmap('viridis'),
        node_color=nx.get_node_attributes(G,'color').values(), 
        with_labels=True,
        #font_weight='bold', 
        #font_color='white',
       )


plt.show()

### Topic Modeling with the concepts

Preparing Document-Term Matrix

All the text documents combined is known as the corpus. To run any mathematical model on text corpus, it is a good practice to convert it into a matrix representation. LDA model looks for repeating term patterns in the entire DT matrix. Python provides many great libraries for text mining practices, “gensim” is one such clean and beautiful library to handle text data. It is scalable, robust and efficient. Following code shows how to convert a corpus into a document-term matrix.

In [ ]:
all_concepts = []
for entry in frage.question_words.values:
    all_concepts.append(entry)
    
# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(all_concepts)
dictionary.filter_extremes(no_below=1, no_above=0.8)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in all_concepts]

Running LDA Model

Next step is to create an object for LDA model and train it on Document-Term matrix. The training also requires few parameters as input which are explained in the above section. The gensim module allows both LDA model estimation from a training corpus and inference of topic distribution on new, unseen documents.

In [ ]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=8, id2word = dictionary, passes=50)

In [ ]:
ldamodel.print_topics(num_topics=8, num_words=7)
#ldamodel.show_topics(10)

In [ ]:
topics_matrix = ldamodel.show_topics(formatted=False, num_words=20)
dic_topics = {num:words for num, words in topics_matrix}
df = pd.DataFrame(dic_topics)
df = df.reindex(sorted(df.columns), axis=1)
df.head(20)
#{num:words for num, words in topics_matrix}

In [ ]:
pyLDAvis.enable_notebook()
p = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)
pyLDAvis.save_html(p, 'lda.html')

Using [Word Embeddings](https://devmount.github.io/GermanWordEmbeddings/#download)